In [10]:
!pip install -U boto3 botocore

In [28]:
import boto3
import os
import json
from botocore.config import Config
import botocore 

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

# modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = "application/json"
contentType = "application/json"

bedrock_runtime = boto3.client(service_name='bedrock-runtime',config=retry_config)


def bedrock_streemer(response):
    stream = response.get('body')
    answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1
    return answer

def call_claude_sonet(prompt, base64_string=None, streaming=True):
    if base64_string is None:
        prompt_config = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                    ],
                }
            ],
        }
    else:
        prompt_config = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/png",
                                "data": base64_string,
                            },
                        },
                        {"type": "text", "text": prompt},
                    ],
                }
            ],
        }


    body = json.dumps(prompt_config)

    modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
    accept = "application/json"
    contentType = "application/json"
    
    if streaming:
        
        response = bedrock_runtime.invoke_model_with_response_stream(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        results=bedrock_streemer(response)
    else:
        response = bedrock_runtime.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())
        results = response_body.get("content")[0].get("text")
    return results
    

# def perform_qa(question, text, streaming=True):
#     """
#     Function to perform a Q&A operation based on the provided text.
#     """
#     prompt = f"Given the following text, answer the question. If the answer is not in the text, 'say you do not know': {question} text: {text} "
#     result = call_claude_sonet(prompt, streaming)
#     return result

In [29]:
prompt = "나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."
call_claude_sonet(prompt)

aaa
생명보험과 손해보험은 보험의 주된 목적과 보장 대상이 다릅니다.

생명보험:
- 사람의 생명과 관련된 위험을 보장합니다. 
- 주요 보장 위험은 사망, 상해, 질병 등입니다.
- 보험금 지급 사유가 발생하면 가입자나 수익자에게 보험금을 지급합니다.
- 대표적인 종류로 정기보험, whole life 보험, 연금보험 등이 있습니다.

손해보험: 
- 재산 손실이나 법적 책임 등 경제적 손해를 보상합니다.
- 주요 보장 위험은 화재, 재해, 사고, 배상책임 등입니다. 
- 보험금은 실제 입은 손해액을 보상하는 것이 원칙입니다.
- 대표적인 종류로 화재보험, 자동차보험, 배상책임보험 등이 있습니다.

간단히 말하면, 생명보험은 개인의 생명 위험을 보장하는 반면, 손해보험은 재물 손실이나 법적 배상책임 등 경제적 손해를 보상하는 보험입니다.

'생명보험과 손해보험은 보험의 주된 목적과 보장 대상이 다릅니다.\n\n생명보험:\n- 사람의 생명과 관련된 위험을 보장합니다. \n- 주요 보장 위험은 사망, 상해, 질병 등입니다.\n- 보험금 지급 사유가 발생하면 가입자나 수익자에게 보험금을 지급합니다.\n- 대표적인 종류로 정기보험, whole life 보험, 연금보험 등이 있습니다.\n\n손해보험: \n- 재산 손실이나 법적 책임 등 경제적 손해를 보상합니다.\n- 주요 보장 위험은 화재, 재해, 사고, 배상책임 등입니다. \n- 보험금은 실제 입은 손해액을 보상하는 것이 원칙입니다.\n- 대표적인 종류로 화재보험, 자동차보험, 배상책임보험 등이 있습니다.\n\n간단히 말하면, 생명보험은 개인의 생명 위험을 보장하는 반면, 손해보험은 재물 손실이나 법적 배상책임 등 경제적 손해를 보상하는 보험입니다.'

In [30]:
def image_to_base64(img):
    import base64
    """Converts a PIL Image or local image file path to a base64 string"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")

In [36]:
prompt = "Provide a caption for this image"
prompt = "100메가 인터넷 요금에 대해 모바일 총액이 64900원 이상일 경우 할인 금액을 알려줘"
prompt = "100메가 인터넷 요금에 대해 모바일 총액이 67000원 이상일 경우 할인 금액을 알려줘"
base64_string = image_to_base64('table.png')
call_claude_sonet(prompt,base64_string)

Reading image from file: table.png
이미지에 제시된 요금표를 보면, 100메가 인터넷 요금은 1,650원입니다. 그리고 모바일 총액이 141,900원 이상일 때 할인금액이 5,500원으로 나와 있습니다. 따라서 모바일 총액이 67,000원 이상일 경우에도 100메가 인터넷 요금에 대한 할인금액은 5,500원입니다.

'이미지에 제시된 요금표를 보면, 100메가 인터넷 요금은 1,650원입니다. 그리고 모바일 총액이 141,900원 이상일 때 할인금액이 5,500원으로 나와 있습니다. 따라서 모바일 총액이 67,000원 이상일 경우에도 100메가 인터넷 요금에 대한 할인금액은 5,500원입니다.'